In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Datenpfad
data_path = Path("/pscratch/sd/t/tbuerger/data/optPhotonSensitiveSurface/MLFormatHomogeneousNCsZylSSD300PMTs")
train_file = data_path / "resum_output_0_train.hdf5"
val_file = data_path / "resum_output_0_validation.hdf5"

# Prüfe Existenz
assert train_file.exists(), f"Train-File nicht gefunden: {train_file}"
assert val_file.exists(), f"Val-File nicht gefunden: {val_file}"

print(f"✓ Dateien gefunden")
print(f"  Train: {train_file}")
print(f"  Val:   {val_file}")

✓ Dateien gefunden
  Train: /pscratch/sd/t/tbuerger/data/optPhotonSensitiveSurface/MLFormatHomogeneousNCsZylSSD300PMTs/resum_output_0_train.hdf5
  Val:   /pscratch/sd/t/tbuerger/data/optPhotonSensitiveSurface/MLFormatHomogeneousNCsZylSSD300PMTs/resum_output_0_validation.hdf5


In [2]:
# Öffne Train-File und analysiere Struktur
with h5py.File(train_file, 'r') as f:
    # Anzahl Neutron Captures
    n_captures_train = len(f['phi']['xNC_mm'])
    
    # Anzahl Voxel
    voxel_ids = list(f['target'].keys())
    n_voxels = len(voxel_ids)
    
    # Beispiel-Voxel Shape
    example_shape = f['target'][voxel_ids[0]].shape
    
    print(f"=== Train Dataset ===")
    print(f"Neutron Captures: {n_captures_train:,}")
    print(f"Anzahl Voxel:     {n_voxels:,}")
    print(f"Target Shape:     {example_shape}")

# Validation Dataset
with h5py.File(val_file, 'r') as f:
    n_captures_val = len(f['phi']['xNC_mm'])
    print(f"\n=== Validation Dataset ===")
    print(f"Neutron Captures: {n_captures_val:,}")

print(f"\n=== Gesamt ===")
print(f"Total Captures:   {n_captures_train + n_captures_val:,}")

=== Train Dataset ===
Neutron Captures: 3,627,360
Anzahl Voxel:     9,583
Target Shape:     (3627360,)

=== Validation Dataset ===
Neutron Captures: 906,138

=== Gesamt ===
Total Captures:   4,533,498


In [3]:
# Anzahl zu sampleder Neutron Captures
N_SAMPLE = 10_000  # Anpassbar

# Stelle sicher, dass nicht mehr gesampelt wird als vorhanden
N_SAMPLE = min(N_SAMPLE, n_captures_train)

# Zufällige Indices (reproduzierbar)
np.random.seed(42)
sample_indices = np.random.choice(n_captures_train, size=N_SAMPLE, replace=False)
sample_indices = np.sort(sample_indices)  # Sortieren für effizienteres HDF5-Lesen

print(f"Sampling {N_SAMPLE:,} von {n_captures_train:,} Captures ({N_SAMPLE/n_captures_train*100:.2f}%)")

Sampling 10,000 von 3,627,360 Captures (0.28%)


In [ ]:
# Lade Voxel-Hits für gesampelte Captures
voxel_hit_sums = []

with h5py.File(train_file, 'r') as f:
    for voxel_id in voxel_ids:
        # Summiere Hits über gesampelte Captures für dieses Voxel
        hits = f['target'][voxel_id][sample_indices]
        total_hits = np.sum(hits)
        voxel_hit_sums.append(total_hits)

voxel_hit_sums = np.array(voxel_hit_sums)

n_total_captures = 10_000_000
n_captures_with_data = n_captures_train
n_captures_zero_hits = n_total_captures - n_captures_with_data

# Anteil der gesampleten 0-Hit-Captures
n_zero_hits_in_sample = int(n_captures_zero_hits * (N_SAMPLE / n_total_captures))

# Erweitere voxel_hit_sums: für jeden Voxel addiere die 0-Hit-Captures
# (haben keinen Effekt auf individuelle Voxel-Sums, nur für Kontext)
print(f"\nInfo: {n_captures_zero_hits:,} Captures haben 0 detektierte Photonen (nicht in HDF5)")

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

# Histogramm mit Binning=1
bins = np.arange(0, voxel_hit_sums.max() + 2) - 0.5  # Zentriere Bins auf Integer-Werte
ax.hist(voxel_hit_sums, bins=bins, edgecolor='black', alpha=0.7)

ax.set_xlabel('Anzahl Hits pro Voxel (summiert über alle gesampleten Captures)', fontsize=12)
ax.set_ylabel('Anzahl Voxel', fontsize=12)
ax.set_title('Verteilung der Voxel-Hit-Counts', fontsize=14, fontweight='bold')
textstr = (f'Anzahl gesampleter Events: {N_SAMPLE:,}\n'
           f'Anzahl Voxel: {n_voxels:,}\n'
           f'Hinweis: {n_captures_zero_hits:,} Captures mit 0 Hits nicht gesampelt')
ax.grid(axis='y', alpha=0.3)

# Annotation
textstr = f'Anzahl gesampleter Events: {N_SAMPLE:,}\nAnzahl Voxel: {n_voxels:,}'
ax.text(0.98, 0.98, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# Statistik
print(f"\n=== Statistik: Hits pro Voxel ===")
print(f"Min:    {voxel_hit_sums.min()}")
print(f"Max:    {voxel_hit_sums.max()}")
print(f"Median: {np.median(voxel_hit_sums):.1f}")
print(f"Mean:   {np.mean(voxel_hit_sums):.1f}")
print(f"Voxel mit 0 Hits: {np.sum(voxel_hit_sums == 0)}")

In [ ]:
# Berechne Gesamthits pro Capture
total_hits_per_capture = []

with h5py.File(train_file, 'r') as f:
    for idx in sample_indices:
        # Summiere Hits über alle Voxel für dieses Capture
        capture_total = 0
        for voxel_id in voxel_ids:
            capture_total += f['target'][voxel_id][idx]
        total_hits_per_capture.append(capture_total)

total_hits_per_capture = np.array(total_hits_per_capture)

# Füge 0-Hit-Captures hinzu
zero_hit_captures_sampled = np.zeros(n_zero_hits_in_sample, dtype=int)
total_hits_per_capture = np.concatenate([total_hits_per_capture, zero_hit_captures_sampled])

# Update N_SAMPLE für korrekte Statistik
N_SAMPLE_TOTAL = len(total_hits_per_capture)
print(f"\nGesamt gesampelte Captures (inkl. 0-Hit): {N_SAMPLE_TOTAL:,}")
# Plot
fig, ax = plt.subplots(figsize=(10, 6))

# Histogramm
bins = np.arange(0, total_hits_per_capture.max() + 2) - 0.5
ax.hist(total_hits_per_capture, bins=bins, edgecolor='black', alpha=0.7, color='steelblue')

ax.set_xlabel('Gesamtzahl Voxel-Hits pro Neutron Capture', fontsize=12)
ax.set_ylabel('Anzahl Neutron Captures', fontsize=12)
ax.set_title('Verteilung der Gesamt-Hits pro Neutron Capture', fontsize=14, fontweight='bold')
textstr = (f'Anzahl gesampleter Events: {N_SAMPLE_TOTAL:,}\n'
           f'davon {n_zero_hits_in_sample:,} mit 0 Hits\n'
           f'Total NC Events: {n_total_captures:,}')
ax.grid(axis='y', alpha=0.3)

# Annotation
textstr = f'Anzahl gesampleter Events: {N_SAMPLE:,}'
ax.text(0.98, 0.98, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.show()

# Statistik
print(f"\n=== Statistik: Hits pro Neutron Capture ===")
print(f"Min:    {total_hits_per_capture.min()}")
print(f"Max:    {total_hits_per_capture.max()}")
print(f"Median: {np.median(total_hits_per_capture):.1f}")
print(f"Mean:   {np.mean(total_hits_per_capture):.1f}")
print(f"Captures mit 0 Hits: {np.sum(total_hits_per_capture == 0)} ({np.sum(total_hits_per_capture == 0)/N_SAMPLE_TOTAL*100:.2f}%)")
print(f"  davon {n_zero_hits_in_sample:,} nicht in HDF5 (keine Photonen detektiert)")
print(f"  und {np.sum(total_hits_per_capture == 0) - n_zero_hits_in_sample:,} in HDF5 mit 0 zugeordneten Hits")

# Quantile
print(f"\nQuantile:")
for q in [0.25, 0.5, 0.75, 0.90, 0.95, 0.99]:
    print(f"  {q*100:5.1f}%: {np.quantile(total_hits_per_capture, q):.0f}")